# RTW MAM Model V1 
# County Data First

In [410]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
import csv
import pickle

In [411]:
#SET DATE FILTERS


prevalence = ['2020-06-21','2020-06-20','2020-06-19','2020-06-18','2020-06-17','2020-06-16','2020-06-15','2020-06-14','2020-06-13','2020-06-12',
              '2020-06-11','2020-06-10','2020-06-09','2020-06-08']

day7_current = ['2020-06-21','2020-06-20','2020-06-19','2020-06-18','2020-06-17','2020-06-16','2020-06-15',]

day7_previous = ['2020-06-14','2020-06-13','2020-06-12',
              '2020-06-11','2020-06-10','2020-06-09','2020-06-08']

curr_test_start = [int(20200621)]
curr_test_end = [int(20200615)]
prev_test_start = [int(20200614)]
prev_test_end = [int(20200608)]

In [412]:
#Creating pop file
#df = pd.read_csv("County4techs_4_15.csv",names=["I", "day", "cases", "county","state","doubling","fips","pop"],
#                dtype={'fips': str})

#df_filtered = df[(df.day==5)]
#print(df.shape)

#pop_df = df_filtered[['fips','county','state','pop' ]]
#print(pop_df.head())

#pickle.dump(pop_df, open("pop_df.sav","wb" ))

In [413]:
#UPDATES NEEDED - bring in base file, within base file add in state 2 digit codes
#Need to add in the NYC FIPS dummy code

#Bring in County Data set
nytimes = "https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv"
counties = pd.read_csv(nytimes,dtype={'fips': str})
print(nyt.head())

#Change to own pop_df location
#pop_df = pickle.load(open("pop_df.sav","rb"))

#print(pop_df.head())
#print(type(pop_df))


#Base file
#base = "https://raw.githubusercontent.com/cbmeco/COVID_MAM/master/base_rtw_v1.csv?token=AF6SWKZK76VX6YZEHQ6ZSD2652GCE"
#base_csv = pd.read_csv(base,dtype={'fips': str})
#print(base_csv.head())


#Want to change this to github csv but for now its Saturday night and I'm alive
pop_df = pd.read_csv("base_rtw_v1.csv",
                dtype={'fips': str})

print(pop_df.head())

         date     county       state   fips  cases  deaths
0  2020-01-21  Snohomish  Washington  53061      1       0
1  2020-01-22  Snohomish  Washington  53061      1       0
2  2020-01-23  Snohomish  Washington  53061      1       0
3  2020-01-24       Cook    Illinois  17031      1       0
4  2020-01-24  Snohomish  Washington  53061      1       0
    fips         County       State state_code Level 6 NBCU Flag  \
0  36061  New York City    New York         NY     NaN         Y   
1  06037    Los Angeles  California         CA     NaN         Y   
2  17031           Cook    Illinois         IL     NaN         Y   
3  48195       Hansford       Texas         TX     NaN       NaN   
4  04013       Maricopa     Arizona         AZ     NaN         Y   

   Tech Ops EE Count    CAE  Cable Stores  Business Services  Total CCC  \
0                0.0    0.0           0.0                0.0        0.0   
1                0.0    0.0           0.0                2.0       98.0   
2           

# Create Active Case Count

In [414]:
#Create new cases field
#Sort df by Fips and Date
counties.sort_values(by=['fips','date'],inplace=True, ascending=True)
print(counties.head())

#Take difference between rows for new case and new death numbers
counties['case_diff'] = counties.cases.diff()
counties['death_diff'] = counties.deaths.diff()


print(counties.head())

             date   county    state   fips  cases  deaths
9480   2020-03-24  Autauga  Alabama  01001      1       0
10832  2020-03-25  Autauga  Alabama  01001      4       0
12361  2020-03-26  Autauga  Alabama  01001      6       0
14021  2020-03-27  Autauga  Alabama  01001      6       0
15801  2020-03-28  Autauga  Alabama  01001      6       0
             date   county    state   fips  cases  deaths  case_diff  \
9480   2020-03-24  Autauga  Alabama  01001      1       0        NaN   
10832  2020-03-25  Autauga  Alabama  01001      4       0        3.0   
12361  2020-03-26  Autauga  Alabama  01001      6       0        2.0   
14021  2020-03-27  Autauga  Alabama  01001      6       0        0.0   
15801  2020-03-28  Autauga  Alabama  01001      6       0        0.0   

       death_diff  
9480          NaN  
10832         0.0  
12361         0.0  
14021         0.0  
15801         0.0  


In [415]:
#Filter for past 14 days
range_14 = counties[counties['date'].isin(prevalence)]
print(range_14.head(15))

              date   county    state   fips  cases  deaths  case_diff  \
215613  2020-06-08  Autauga  Alabama  01001    273       5        7.0   
218625  2020-06-09  Autauga  Alabama  01001    283       5       10.0   
221641  2020-06-10  Autauga  Alabama  01001    296       6       13.0   
224663  2020-06-11  Autauga  Alabama  01001    316       6       20.0   
227686  2020-06-12  Autauga  Alabama  01001    327       6       11.0   
230711  2020-06-13  Autauga  Alabama  01001    335       6        8.0   
233737  2020-06-14  Autauga  Alabama  01001    361       6       26.0   
236764  2020-06-15  Autauga  Alabama  01001    368       6        7.0   
239795  2020-06-16  Autauga  Alabama  01001    373       7        5.0   
242827  2020-06-17  Autauga  Alabama  01001    375       7        2.0   
245859  2020-06-18  Autauga  Alabama  01001    400       8       25.0   
248894  2020-06-19  Autauga  Alabama  01001    411       8       11.0   
251934  2020-06-20  Autauga  Alabama  01001    431 

In [416]:
#Group by FIPS, sum up new Cases for prevalence - for 7 day averages will take mean after filtering for 7 days
prevalence_grouped = range_14.groupby(['fips'], as_index=False).sum()

print(prevalence_grouped.head(25))

     fips  cases  deaths  case_diff  death_diff
0   01001   4983      94      168.0         4.0
1   01003   5330     126      101.0         0.0
2   01005   3313      14       78.0         0.0
3   01007   1513      14       47.0         0.0
4   01009   1556      14       75.0         0.0
5   01011   4056     129       94.0         3.0
6   01013   7376     350      125.0         4.0
7   01015   2815      48       29.0         1.0
8   01017   6496     368      131.0         1.0
9   01019    712      69       15.0         2.0
10  01021   1962      32       51.0         1.0
11  01023   2424     154       30.0         2.0
12  01025   2622      46       59.0         2.0
13  01027    422      28        3.0         0.0
14  01029    292      14        4.0         0.0
15  01031   4000      14       68.0         0.0
16  01033   3692      69       90.0         1.0
17  01035   1363      14       89.0         0.0
18  01037    630      14       10.0         0.0
19  01039   2208      14      135.0     

In [417]:
#Join in Prevalence to 
prevalence_w_pop = pd.merge(pop_df,
                 prevalence_grouped[['fips','case_diff']],
                 on='fips', 
                how='left')
print(prevalence_w_pop.head(10))

    fips         County       State state_code Level 6 NBCU Flag  \
0  36061  New York City    New York         NY     NaN         Y   
1  06037    Los Angeles  California         CA     NaN         Y   
2  17031           Cook    Illinois         IL     NaN         Y   
3  48195       Hansford       Texas         TX     NaN       NaN   
4  04013       Maricopa     Arizona         AZ     NaN         Y   
5  06073      San Diego  California         CA     NaN         Y   
6  06059         Orange  California         CA     NaN       NaN   
7  12086     Miami-Dade     Florida         FL     NaN         Y   
8  48107         Crosby       Texas         TX     NaN       NaN   
9  06065      Riverside  California         CA     NaN       NaN   

   Tech Ops EE Count    CAE  Cable Stores  Business Services  Total CCC  \
0                0.0    0.0           0.0                0.0        0.0   
1                0.0    0.0           0.0                2.0       98.0   
2              965.0  356.

In [418]:
prevalence_w_pop['active_cases_100k'] = ((prevalence_w_pop['case_diff']/prevalence_w_pop['pop'])*100000)
print(prevalence_w_pop.head())

    fips         County       State state_code Level 6 NBCU Flag  \
0  36061  New York City    New York         NY     NaN         Y   
1  06037    Los Angeles  California         CA     NaN         Y   
2  17031           Cook    Illinois         IL     NaN         Y   
3  48195       Hansford       Texas         TX     NaN       NaN   
4  04013       Maricopa     Arizona         AZ     NaN         Y   

   Tech Ops EE Count    CAE  Cable Stores  Business Services  Total CCC  \
0                0.0    0.0           0.0                0.0        0.0   
1                0.0    0.0           0.0                2.0       98.0   
2              965.0  356.0         201.0              146.0     3640.0   
3                0.0    0.0           0.0                0.0        0.0   
4                0.0    0.0           0.0                3.0        9.0   

        Geo Area %Retail Closed in County Updated 5/26      Division  \
0            NaN                                   NaN     Northeast

# Create 7 Day Rolling Avgs for Cases & Deaths

In [419]:
#Filter Diff datasets for current and previous 7 day periods

#join prev and current together with pop

range_current7 = counties[counties['date'].isin(day7_current)]
range_prev7 = counties[counties['date'].isin(day7_previous)]

print(range_current7.head())

              date   county    state   fips  cases  deaths  case_diff  \
236764  2020-06-15  Autauga  Alabama  01001    368       6        7.0   
239795  2020-06-16  Autauga  Alabama  01001    373       7        5.0   
242827  2020-06-17  Autauga  Alabama  01001    375       7        2.0   
245859  2020-06-18  Autauga  Alabama  01001    400       8       25.0   
248894  2020-06-19  Autauga  Alabama  01001    411       8       11.0   

        death_diff  
236764         0.0  
239795         1.0  
242827         0.0  
245859         1.0  
248894         0.0  


In [420]:
#Group by with mean
current_grouped = range_current7.groupby(['fips'],as_index=False).mean()
previous_grouped = range_prev7.groupby(['fips'], as_index=False).mean()

current_grouped['curr7_case'] = current_grouped['case_diff']
current_grouped['curr7_death'] = current_grouped['death_diff']
previous_grouped['prev7_case'] = previous_grouped['case_diff']
previous_grouped['prev7_death'] = previous_grouped['death_diff']
print(current_grouped.head())
print(previous_grouped.head())

    fips       cases    deaths  case_diff  death_diff  curr7_case  curr7_death
0  01001  398.857143  7.714286  10.428571    0.428571   10.428571     0.428571
1  01003  404.000000  9.000000   7.285714    0.000000    7.285714     0.000000
2  01005  258.142857  1.000000   5.285714    0.000000    5.285714     0.000000
3  01007  120.714286  1.000000   2.857143    0.000000    2.857143     0.000000
4  01009  131.571429  1.000000   5.285714    0.000000    5.285714     0.000000
    fips       cases    deaths  case_diff  death_diff  prev7_case  prev7_death
0  01001  313.000000  5.714286  13.571429    0.142857   13.571429     0.142857
1  01003  357.428571  9.000000   7.142857    0.000000    7.142857     0.000000
2  01005  215.142857  1.000000   5.857143    0.000000    5.857143     0.000000
3  01007   95.428571  1.000000   3.857143    0.000000    3.857143     0.000000
4  01009   90.714286  1.000000   5.428571    0.000000    5.428571     0.000000


In [421]:
#Merge both sets with population
current_w_pop = pd.merge(pop_df,
                 current_grouped[['fips','curr7_case', 'curr7_death']],
                 on='fips', 
                how='left')
print(current_w_pop.head(10))

previous_w_pop = pd.merge(pop_df,
                 previous_grouped[['fips','prev7_case', 'prev7_death']],
                 on='fips', 
                how='left')
print(previous_w_pop.head(10))

    fips         County       State state_code Level 6 NBCU Flag  \
0  36061  New York City    New York         NY     NaN         Y   
1  06037    Los Angeles  California         CA     NaN         Y   
2  17031           Cook    Illinois         IL     NaN         Y   
3  48195       Hansford       Texas         TX     NaN       NaN   
4  04013       Maricopa     Arizona         AZ     NaN         Y   
5  06073      San Diego  California         CA     NaN         Y   
6  06059         Orange  California         CA     NaN       NaN   
7  12086     Miami-Dade     Florida         FL     NaN         Y   
8  48107         Crosby       Texas         TX     NaN       NaN   
9  06065      Riverside  California         CA     NaN       NaN   

   Tech Ops EE Count    CAE  Cable Stores  Business Services  Total CCC  \
0                0.0    0.0           0.0                0.0        0.0   
1                0.0    0.0           0.0                2.0       98.0   
2              965.0  356.

In [422]:
current_w_pop['cur_7rollavg_cases'] = ((current_w_pop['curr7_case']/current_w_pop['pop'])*100000)
current_w_pop['cur_7rollavg_deaths'] = ((current_w_pop['curr7_death']/current_w_pop['pop'])*100000)

In [423]:
previous_w_pop['prev_7rollavg_cases'] = ((previous_w_pop['prev7_case']/previous_w_pop['pop'])*100000)
previous_w_pop['prev_7rollavg_deaths'] = ((previous_w_pop['prev7_death']/previous_w_pop['pop'])*100000)

In [424]:
#Merge with prevalence dataset
prev_curr = pd.merge(prevalence_w_pop,
                 current_w_pop[['fips','cur_7rollavg_cases', 'cur_7rollavg_deaths','curr7_case','curr7_death']],
                 on='fips', 
                how='left')
print(prev_curr.head(10))

    fips         County       State state_code Level 6 NBCU Flag  \
0  36061  New York City    New York         NY     NaN         Y   
1  06037    Los Angeles  California         CA     NaN         Y   
2  17031           Cook    Illinois         IL     NaN         Y   
3  48195       Hansford       Texas         TX     NaN       NaN   
4  04013       Maricopa     Arizona         AZ     NaN         Y   
5  06073      San Diego  California         CA     NaN         Y   
6  06059         Orange  California         CA     NaN       NaN   
7  12086     Miami-Dade     Florida         FL     NaN         Y   
8  48107         Crosby       Texas         TX     NaN       NaN   
9  06065      Riverside  California         CA     NaN       NaN   

   Tech Ops EE Count    CAE  Cable Stores  Business Services     ...       \
0                0.0    0.0           0.0                0.0     ...        
1                0.0    0.0           0.0                2.0     ...        
2              965.0

In [425]:
county_set = pd.merge(prev_curr,
                 previous_w_pop[['fips','prev_7rollavg_cases', 'prev_7rollavg_deaths','prev7_case','prev7_death']],
                 on='fips', 
                how='left')
print(county_set.head(10))

    fips         County       State state_code Level 6 NBCU Flag  \
0  36061  New York City    New York         NY     NaN         Y   
1  06037    Los Angeles  California         CA     NaN         Y   
2  17031           Cook    Illinois         IL     NaN         Y   
3  48195       Hansford       Texas         TX     NaN       NaN   
4  04013       Maricopa     Arizona         AZ     NaN         Y   
5  06073      San Diego  California         CA     NaN         Y   
6  06059         Orange  California         CA     NaN       NaN   
7  12086     Miami-Dade     Florida         FL     NaN         Y   
8  48107         Crosby       Texas         TX     NaN       NaN   
9  06065      Riverside  California         CA     NaN       NaN   

   Tech Ops EE Count    CAE  Cable Stores  Business Services     ...       \
0                0.0    0.0           0.0                0.0     ...        
1                0.0    0.0           0.0                2.0     ...        
2              965.0

In [426]:
#Create indicators for rolling averages
county_set['roll7case_diff'] = ((county_set['cur_7rollavg_cases']-county_set['prev_7rollavg_cases'])/county_set['prev_7rollavg_cases'])
county_set['roll7death_diff'] = ((county_set['cur_7rollavg_deaths']-county_set['prev_7rollavg_deaths'])/county_set['prev_7rollavg_deaths'])

print(county_set.head())

    fips         County       State state_code Level 6 NBCU Flag  \
0  36061  New York City    New York         NY     NaN         Y   
1  06037    Los Angeles  California         CA     NaN         Y   
2  17031           Cook    Illinois         IL     NaN         Y   
3  48195       Hansford       Texas         TX     NaN       NaN   
4  04013       Maricopa     Arizona         AZ     NaN         Y   

   Tech Ops EE Count    CAE  Cable Stores  Business Services       ...         \
0                0.0    0.0           0.0                0.0       ...          
1                0.0    0.0           0.0                2.0       ...          
2              965.0  356.0         201.0              146.0       ...          
3                0.0    0.0           0.0                0.0       ...          
4                0.0    0.0           0.0                3.0       ...          

   cur_7rollavg_cases cur_7rollavg_deaths   curr7_case curr7_death  \
0                 NaN             

In [427]:
#Write to CSV
#county_set.to_csv('CountyMAMV2_618.csv') 

# State Level Data

In [428]:
# Pull in NYTimes state data, diff, do same 7 day rolling avg counts
# pull in COVID tracking test data. calculate pos % then rolling avgs
# Manually pull in Rt, might drop this metric

In [429]:
#Bring in state level NYT Data
nytimes_state = "https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-states.csv"
state= pd.read_csv(nytimes_state)
print(nyt_state.head())

         date       state  fips  cases  deaths
0  2020-01-21  Washington    53      1       0
1  2020-01-22  Washington    53      1       0
2  2020-01-23  Washington    53      1       0
3  2020-01-24    Illinois    17      1       0
4  2020-01-24  Washington    53      1       0


In [430]:
#Create new cases field
#Sort df by Fips and Date
state['State'] = state['state']
state.sort_values(by=['fips','date'],inplace=True, ascending=True)
print(state.head())

#Take difference between rows for new case and new death numbers
state['case_diff'] = state.cases.diff()
state['death_diff'] = state.deaths.diff()
print(state.head())

           date    state  fips  cases  deaths    State
586  2020-03-13  Alabama     1      6       0  Alabama
637  2020-03-14  Alabama     1     12       0  Alabama
689  2020-03-15  Alabama     1     23       0  Alabama
742  2020-03-16  Alabama     1     29       0  Alabama
795  2020-03-17  Alabama     1     39       0  Alabama
           date    state  fips  cases  deaths    State  case_diff  death_diff
586  2020-03-13  Alabama     1      6       0  Alabama        NaN         NaN
637  2020-03-14  Alabama     1     12       0  Alabama        6.0         0.0
689  2020-03-15  Alabama     1     23       0  Alabama       11.0         0.0
742  2020-03-16  Alabama     1     29       0  Alabama        6.0         0.0
795  2020-03-17  Alabama     1     39       0  Alabama       10.0         0.0


In [431]:
##CREATE State Prevalence measure
#Filter for past 14 days
state_range_14 = state[state['date'].isin(prevalence)]
print(state_range_14.head())

            date    state  fips  cases  deaths    State  case_diff  death_diff
5349  2020-06-08  Alabama     1  20925     718  Alabama      425.0        26.0
5404  2020-06-09  Alabama     1  21422     729  Alabama      497.0        11.0
5459  2020-06-10  Alabama     1  21989     744  Alabama      567.0        15.0
5514  2020-06-11  Alabama     1  22845     755  Alabama      856.0        11.0
5569  2020-06-12  Alabama     1  23710     769  Alabama      865.0        14.0


In [432]:
#create state group for prevalence

state_prevalence_grouped = state_range_14.groupby(['State'], as_index=False).sum()

state_prevalence_grouped['state_active'] = state_prevalence_grouped['case_diff']
print(state_prevalence_grouped.head())

        State  fips    cases  deaths  case_diff  death_diff  state_active
0     Alabama    14   358381   10919     9521.0       147.0        9521.0
1      Alaska    28    10264     135      271.0         2.0         271.0
2     Arizona    56   532527   16887    25742.0       302.0       25742.0
3    Arkansas    70   176123    2622     6135.0        71.0        6135.0
4  California    84  2175865   71799    46810.0       864.0       46810.0


In [433]:
#Take Current and previous rolling averages
state_range_current7 = state[state['date'].isin(day7_current)]
state_range_prev7 = state[state['date'].isin(day7_previous)]

print(state_range_current7.head(8))

            date    state  fips  cases  deaths    State  case_diff  death_diff
5734  2020-06-15  Alabama     1  26272     774  Alabama      657.0         1.0
5789  2020-06-16  Alabama     1  26912     785  Alabama      640.0        11.0
5844  2020-06-17  Alabama     1  27312     790  Alabama      400.0         5.0
5899  2020-06-18  Alabama     1  28206     810  Alabama      894.0        20.0
5954  2020-06-19  Alabama     1  29002     822  Alabama      796.0        12.0
6009  2020-06-20  Alabama     1  29549     838  Alabama      547.0        16.0
6064  2020-06-21  Alabama     1  30021     839  Alabama      472.0         1.0
5735  2020-06-15   Alaska     2    738      10   Alaska        4.0         0.0


In [434]:
#Groupby Means
#Group by with mean
state_current_grouped = state_range_current7.groupby(['State'],as_index=False).mean()
state_previous_grouped = state_range_prev7.groupby(['State'], as_index=False).mean()

state_current_grouped['state_curr7_case'] = state_current_grouped['case_diff']
state_previous_grouped['state_prev7_case'] = state_previous_grouped['case_diff']
state_current_grouped['state_curr7_death'] = state_current_grouped['death_diff']
state_previous_grouped['state_prev7_death'] = state_previous_grouped['death_diff']

print(state_current_grouped.head())
print(state_previous_grouped.head())

        State  fips          cases       deaths    case_diff  death_diff  \
0     Alabama   1.0   28182.000000   808.285714   629.428571    9.428571   
1      Alaska   2.0     797.857143    10.000000    17.857143    0.000000   
2     Arizona   4.0   44424.428571  1284.285714  2387.571429   22.285714   
3    Arkansas   5.0   14139.428571   205.428571   437.142857    6.571429   
4  California   6.0  167116.142857  5343.000000  3693.428571   61.142857   

   state_curr7_case  state_curr7_death  
0        629.428571           9.428571  
1         17.857143           0.000000  
2       2387.571429          22.285714  
3        437.142857           6.571429  
4       3693.428571          61.142857  
        State  fips          cases       deaths    case_diff  death_diff  \
0     Alabama   1.0   23015.285714   751.571429   730.714286   11.571429   
1      Alaska   2.0     668.428571     9.285714    20.857143    0.285714   
2     Arizona   4.0   31650.857143  1128.142857  1289.857143   20.857

In [435]:
#Merge both sets with base file

state_active= pd.merge(county_set,
                 state_prevalence_grouped[['State','state_active',
                                       ]],
                 on='State', 
                how='left')
print(state_current_w_pop.head(10))

state_current_w_pop = pd.merge(state_active,
                 state_current_grouped[['State','state_curr7_case',
                                       'state_curr7_death']],
                 on='State', 
                how='left')
print(state_current_w_pop.head(10))

county_state_df = pd.merge(state_current_w_pop,
                 state_previous_grouped[['State','state_prev7_case','state_prev7_death']],
                 on='State', 
                how='left')
print(state_previous_w_pop.head(10))

    fips         County       State state_code Level 6 NBCU Flag  \
0  36061  New York City    New York         NY     NaN         Y   
1  06037    Los Angeles  California         CA     NaN         Y   
2  17031           Cook    Illinois         IL     NaN         Y   
3  48195       Hansford       Texas         TX     NaN       NaN   
4  04013       Maricopa     Arizona         AZ     NaN         Y   
5  06073      San Diego  California         CA     NaN         Y   
6  06059         Orange  California         CA     NaN       NaN   
7  12086     Miami-Dade     Florida         FL     NaN         Y   
8  48107         Crosby       Texas         TX     NaN       NaN   
9  06065      Riverside  California         CA     NaN       NaN   

   Tech Ops EE Count    CAE  Cable Stores  Business Services  \
0                0.0    0.0           0.0                0.0   
1                0.0    0.0           0.0                2.0   
2              965.0  356.0         201.0              146.

In [436]:
#Create state rolling avg indicator
#####NEED TO ADD IN STATE POP to create per 100k 

county_state_df['state_curr7_case100k'] = ((county_state_df['state_curr7_case']/county_state_df['state_pop'])*100000)
county_state_df['state_curr7_death100k'] = ((county_state_df['state_curr7_death']/county_state_df['state_pop'])*100000)
county_state_df['state_prev7_case100k'] = ((county_state_df['state_prev7_case']/county_state_df['state_pop'])*100000)
county_state_df['state_prev7_death100k'] = ((county_state_df['state_prev7_death']/county_state_df['state_pop'])*100000)


county_state_df['state_roll7case_diff'] = ((county_state_df['state_curr7_case100k']-county_state_df['state_prev7_case100k'])/county_state_df['state_prev7_case100k'])
county_state_df['state_roll7death_diff'] = ((county_state_df['state_curr7_death100k']-county_state_df['state_prev7_death100k'])/county_state_df['state_prev7_death100k'])
county_state_df['state_prevalence_per100k'] = ((county_state_df['state_active']/county_state_df['state_pop'])*100000)

print(county_state_df.head())

    fips         County       State state_code Level 6 NBCU Flag  \
0  36061  New York City    New York         NY     NaN         Y   
1  06037    Los Angeles  California         CA     NaN         Y   
2  17031           Cook    Illinois         IL     NaN         Y   
3  48195       Hansford       Texas         TX     NaN       NaN   
4  04013       Maricopa     Arizona         AZ     NaN         Y   

   Tech Ops EE Count    CAE  Cable Stores  Business Services  \
0                0.0    0.0           0.0                0.0   
1                0.0    0.0           0.0                2.0   
2              965.0  356.0         201.0              146.0   
3                0.0    0.0           0.0                0.0   
4                0.0    0.0           0.0                3.0   

             ...             state_curr7_death state_prev7_case  \
0            ...                     39.857143       745.285714   
1            ...                     61.142857      2993.714286   
2    

# COVID TRACKING PROJ DATA

In [437]:
#########################################
#Bring in COVID TRACKING data
url_all = "https://covidtracking.com/api/v1/states/daily.csv"
testing_all = pd.read_csv(url_all)
testing = testing_all[['date','state','positiveIncrease','negativeIncrease', 'totalTestResultsIncrease']]
testing['state_code'] = testing['state']

print(testing.head())
print(testing.dtypes)

       date state  positiveIncrease  negativeIncrease  \
0  20200621    AK                12              1121   
1  20200621    AL               472              4683   
2  20200621    AR                 0                 0   
3  20200621    AS                 0                 0   
4  20200621    AZ              2592             11872   

   totalTestResultsIncrease state_code  
0                      1133         AK  
1                      5155         AL  
2                         0         AR  
3                         0         AS  
4                     14464         AZ  
date                         int64
state                       object
positiveIncrease             int64
negativeIncrease             int64
totalTestResultsIncrease     int64
state_code                  object
dtype: object


C:\Users\ccolam127\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [438]:
curr_testing = testing[(testing['date']>=curr_test_end) & (testing['date']<=curr_test_start)]
curr_testing['curr_pos_%'] = (curr_testing['positiveIncrease']/curr_testing['totalTestResultsIncrease'])
print(curr_testing.head(10))

prev_testing = testing[(testing['date']>=prev_test_end) & (testing['date']<=prev_test_start)]
prev_testing['prev_pos_%'] = (prev_testing['positiveIncrease']/prev_testing['totalTestResultsIncrease'])
print(prev_testing.head())

       date state  positiveIncrease  negativeIncrease  \
0  20200621    AK                12              1121   
1  20200621    AL               472              4683   
2  20200621    AR                 0                 0   
3  20200621    AS                 0                 0   
4  20200621    AZ              2592             11872   
5  20200621    CA              4515             80329   
6  20200621    CO               162              5155   
7  20200621    CT                40              3733   
8  20200621    DC                36              1533   
9  20200621    DE                94              2464   

   totalTestResultsIncrease state_code  curr_pos_%  
0                      1133         AK    0.010591  
1                      5155         AL    0.091562  
2                         0         AR         NaN  
3                         0         AS         NaN  
4                     14464         AZ    0.179204  
5                     84844         CA    0.053215  
6

C:\Users\ccolam127\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\ccolam127\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [439]:

#Group by with mean
curr_test_grouped = curr_testing.groupby(['state_code'],as_index=False).mean()
prev_test_grouped = prev_testing.groupby(['state_code'], as_index=False).mean()

print(curr_test_grouped.head())
print(prev_test_grouped.head())

  state_code        date  positiveIncrease  negativeIncrease  \
0         AK  20200618.0         13.428571       2070.000000   
1         AL  20200618.0        629.428571       6116.714286   
2         AR  20200618.0        377.285714       6086.285714   
3         AS  20200618.0          0.000000          0.000000   
4         AZ  20200618.0       2385.571429       9285.714286   

   totalTestResultsIncrease  curr_pos_%  
0               2083.428571    0.007055  
1               6746.142857    0.108375  
2               6463.571429    0.083086  
3                  0.000000         NaN  
4              11671.285714    0.203062  
  state_code        date  positiveIncrease  negativeIncrease  \
0         AK  20200611.0         16.714286       1206.571429   
1         AL  20200611.0        730.714286       4682.000000   
2         AR  20200611.0        439.285714       4805.857143   
3         AS  20200611.0          0.000000          0.000000   
4         AZ  20200611.0       1257.428571 

In [444]:
#Merge to County Set

curr_merge = pd.merge(county_state_df,
                 curr_test_grouped[['state_code','curr_pos_%']],
                 on='state_code', 
                how='left')
print(curr_merge.head())

mam_v2 = pd.merge(curr_merge,
                 prev_test_grouped[['state_code','prev_pos_%']],
                 on='state_code', 
                how='left')


mam_v2['testing_diff'] = (mam_v2['curr_pos_%'] - mam_v2['prev_pos_%'])
print(mam_v2.head())

    fips         County       State state_code Level 6 NBCU Flag  \
0  36061  New York City    New York         NY     NaN         Y   
1  06037    Los Angeles  California         CA     NaN         Y   
2  17031           Cook    Illinois         IL     NaN         Y   
3  48195       Hansford       Texas         TX     NaN       NaN   
4  04013       Maricopa     Arizona         AZ     NaN         Y   

   Tech Ops EE Count    CAE  Cable Stores  Business Services     ...      \
0                0.0    0.0           0.0                0.0     ...       
1                0.0    0.0           0.0                2.0     ...       
2              965.0  356.0         201.0              146.0     ...       
3                0.0    0.0           0.0                0.0     ...       
4                0.0    0.0           0.0                3.0     ...       

   state_prev7_case state_prev7_death state_curr7_case100k  \
0        745.285714         60.285714             3.273632   
1       29

In [441]:
####################################################

####### CREATING INDICATOR/SCORES SATURDAY NIGHT!##

county_filtered = mam_v2[(mam_v2['active_cases_100k']>=0)]


#print(county_filtered.active_cases_100k.describe())
#print(county_filtered.roll7case_diff.describe())
#print(county_filtered.roll7death_diff.describe())
#print(county_filtered.state_roll7case_diff.describe())
print(county_filtered.state_prevalence_per100k.describe())


count    2854.000000
mean       96.540852
std        50.148093
min         8.917410
25%        56.137176
50%        85.060740
75%       132.117299
max       348.878782
Name: state_prevalence_per100k, dtype: float64


In [442]:
#Updated 6/22

def active(n):
    if 0<=n<=49.48: score = 0
    elif 49.49<=n<=114.05: score = 1
    elif 114.06<=n<=100000: score = 2
    else: score = 0
    return(score)



def roll(n):
    if -1<=n<=-0.10: score = 0
    elif -0.10<=n<=0.10: score = 1
    elif 0.16<=n<=100: score = 2
    else: score = 0
    return(score)

def fatal(n):
    if n>=0.05: score = 4
    else: score = 0
    return(score)

def test(n):
    if -2<=n<=-.01: score = 0
    elif -.011111<=n<=.01: score = 1
    elif 0.01111 <=n<= 100: score =2
    else: score = 0
    return(score)


county_filtered['County_Active_100k_Score'] = county_filtered['active_cases_100k'].apply(active)
county_filtered['State_Active_100k_Score'] = county_filtered['state_prevalence_per100k'].apply(active)
county_filtered['County_Roll_Score'] = county_filtered['roll7case_diff'].apply(roll)
county_filtered['State_Roll_Score'] = county_filtered['state_roll7case_diff'].apply(roll)
county_filtered['County_Fatality_Roll_Score'] = county_filtered['roll7death_diff'].apply(fatal)
county_filtered['State_Fatality_Roll_Score'] = county_filtered['state_roll7death_diff'].apply(fatal)
county_filtered['State_Testing_Score'] = county_filtered['testing_diff'].apply(test)


print(county_filtered.head())

    fips       County       State state_code Level 6 NBCU Flag  \
1  06037  Los Angeles  California         CA     NaN         Y   
2  17031         Cook    Illinois         IL     NaN         Y   
3  48195     Hansford       Texas         TX     NaN       NaN   
4  04013     Maricopa     Arizona         AZ     NaN         Y   
5  06073    San Diego  California         CA     NaN         Y   

   Tech Ops EE Count    CAE  Cable Stores  Business Services  \
1                0.0    0.0           0.0                2.0   
2              965.0  356.0         201.0              146.0   
3                0.0    0.0           0.0                0.0   
4                0.0    0.0           0.0                3.0   
5                0.0    0.0           0.0                0.0   

          ...           curr_pos_% prev_pos_% testing_diff  \
1         ...             0.048016   0.045966     0.002050   
2         ...             0.026084   0.039487    -0.013402   
3         ...             0.1165

C:\Users\ccolam127\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\ccolam127\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\ccolam127\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in

In [447]:
county_filtered['State_Composite_Score'] =  (county_filtered['State_Active_100k_Score'] + county_filtered['State_Roll_Score']+ county_filtered['State_Testing_Score'])


county_filtered['County_Composite_Score'] = (county_filtered['County_Roll_Score'] + county_filtered['County_Active_100k_Score'])



In [449]:
def county(n):
    if 0<=n<=.99: score = "Minimal"
    elif 0.1<=n<=1: score = "Low"
    elif 1<n<=2: score = "Moderate"
    elif 2<n<=3: score = "Severe"
    elif 4<=n<= 100: score = "Extreme"
    else: score = 0
    return(score)


def state(n):
    if 0<=n<=.99: score = "Minimal"
    elif 0.1<=n<=1: score = "Low"
    elif 2<=n<=3: score = "Moderate"
    elif 4<=n<=5: score = "Severe"
    elif 6 <=n<= 100: score = "Extreme"
    else: score = 0
    return(score)


county_filtered['County_Level'] = county_filtered['County_Composite_Score'].apply(county)
county_filtered['State_Level'] = county_filtered['State_Composite_Score'].apply(state)
print(county_filtered.head())



    fips       County       State state_code Level 6 NBCU Flag  \
1  06037  Los Angeles  California         CA     NaN         Y   
2  17031         Cook    Illinois         IL     NaN         Y   
3  48195     Hansford       Texas         TX     NaN       NaN   
4  04013     Maricopa     Arizona         AZ     NaN         Y   
5  06073    San Diego  California         CA     NaN         Y   

   Tech Ops EE Count    CAE  Cable Stores  Business Services     ...       \
1                0.0    0.0           0.0                2.0     ...        
2              965.0  356.0         201.0              146.0     ...        
3                0.0    0.0           0.0                0.0     ...        
4                0.0    0.0           0.0                3.0     ...        
5                0.0    0.0           0.0                0.0     ...        

   State_Active_100k_Score County_Roll_Score State_Roll_Score  \
1                        2                 0                2   
2         

In [450]:
county_filtered.to_csv('CountyMAMV3_622.csv') 